In [1]:
import numpy as np
import torch
import functools
import time
import matplotlib.pyplot as plt

In [2]:
def poly(*a):
    """
    :param a: a list of coefficients starting with the coefficient of the highest degree down to 0.
    :return: same as numpy.poly1d(a)
    """
    return np.poly1d(a)

def NOISY(noise):
    def decorate(function):
        def wrapper(*args, **kwargs):
            result = function(*args, **kwargs)
            result+=np.random.randn()*noise
            return result
        return wrapper
    return decorate

In [3]:
def find_roots(p: np.poly1d, start, end, maxerr=0.001):
    global base_poly_roots_finders
    coefs = p.coefficients
    d = p.order
    if d < len(base_poly_roots_finders):
        return base_poly_roots_finders[d](start, end, *coefs)
    dp_roots = find_roots(np.polyder(p), start, end)
    return find_roots_from_der_roots(p, start, dp_roots, end, maxerr)

def _find_roots_poly_constant(start, end, constant):
    if constant == 0:
        raise Exception("Zero polynomial has infinite amount roots")
    return []
def _find_roots_poly_linear_line(start, end, slope, constant):
    if slope == 0:
        return _find_roots_poly_constant(start, end, constant)
    root = (-constant) / slope
    return _roots_list(start, end, root)
def _find_roots_poly_quadratic(start, end, a, b, c):
    if a == 0:
        return _find_roots_poly_linear_line(start, end, b ,c)
    return _find_roots_poly_quadratic_core(a, b, c, end, start)

def _find_roots_poly_quadratic_core(a, b, c, end, start):
    delta = (b ** 2) - 4 * a * c
    if delta < 0:
        return []
    elif delta == 0:
        root = -b / (2 * a)
        return _roots_list(start, end, root)
    else:
        return _find_roots_poly_quadratic_2_roots(a, b, delta, end, start)

def _find_roots_poly_quadratic_2_roots(a, b, delta, end, start):
    delta = np.sqrt(delta)
    root0 = (-b + delta) / (2 * a)
    root1 = (-b - delta) / (2 * a)
    root0, root1 = min(root0, root1), max(root0, root1)
    return _roots_list(end, start, root0, root1)


base_poly_roots_finders = [
    _find_roots_poly_constant,
    _find_roots_poly_linear_line,
    _find_roots_poly_quadratic
]

def _roots_list(end, start, *roots):
    return [root for root in roots if start <= root <= end]

def find_poly_roots(p, a, dp_roots, b, maxerr):
    roots = []

    prev_a = a
    p_prev_a = p(a)
    if is_root(p_prev_a, maxerr):
        roots.append(prev_a)

    for i in range(len(dp_roots)):
        dpr = dp_roots[i]
        p_dpr = p(dpr)
        if is_root(p_dpr, maxerr):
            roots.append(dpr)
        elif (p_prev_a < 0) != (p_dpr < 0):
            roots.append(regula_falsi(p, prev_a, dpr, maxerr))
        else:
            pass
        prev_a = dpr
        p_prev_a = p_dpr

    p_b = p(b)
    if is_root(p_b, maxerr):
        roots.append(b)
    elif (p_prev_a < 0) != (p_b < 0):
        roots.append(regula_falsi(p, prev_a, b, maxerr))
    else:
        pass
    return roots

def is_root(y, maxerr):
    return abs(y) <= maxerr

def regula_falsi(f, a, b, maxerr):
    c = b
    fc = fb = f(b)
    while not is_root(fc, maxerr):
        c = b - fb * ((b - a) / (fb - f(a)))
        fc = f(c)
        if (fc < 0) != (fb < 0):
            a = c
        else:
            b = c
            fb = fc
    return c


def poly_to_string(p):
    coefficients = p.coefficients
    highest = coefficients[0]
    n = p.order

    s = ""
    if highest == -1:
        s += "-"
    elif highest == 1:
        pass
    else:
        s += str(highest)
    if n >= 10:
        s += f"x^{{{n}}}"
    elif n > 1:
        s += f"x^{n}"
    elif n == 1:
        s += "x"
    for k in range(1, n - 1):
        coef_string = coef_to_string(coefficients[k])
        if coef_string == "":
            continue
        s += f"{coef_string}x^{n-k}"

    if n > 1:
        coef_string = coef_to_string(coefficients[n - 1])
        if coef_string != "":
            s += f"{coef_string}x"
    if n > 0:
        coef = coefficients[n]
        s += f" {'-' if coef < 0 else '+'} {abs(coef)}"
    return s


def coef_to_string(coef):
    s = ""
    if coef != 0:
        s += f" {'-' if coef < 0 else '+'} "
        if abs(coef) != 1:
            s += str(abs(coef))
    return s

In [4]:
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt(((x1 - x2) ** 2) + ((y1 - y2) ** 2))

def convert_torch_to_numpy(l):
    return [np.float64(x) for x in l]

class Assignment4A:
    def __init__(self):
        """
        Here goes any one time calculation that need to be made before 
        solving the assignment for specific functions. 
        """
        self.Ms = [self.build_M(d) for d in range(13)]

    def fit(self, f: callable, a: float, b: float, d:int, maxtime: float) -> callable:
        """
        Build a function that accurately fits the noisy data points sampled from
        some closed shape. 
        
        Parameters
        ----------
        f : callable. 
            A function which returns an approximate (noisy) Y value given X. 
        a: float
            Start of the fitting range
        b: float
            End of the fitting range
        d: int 
            The expected degree of a polynomial matching f
        maxtime : float
            This function returns after at most maxtime seconds. 

        Returns
        -------
        a function:float->float that fits f between a and b
        """
        start_time = time.time()
        n = 1000
        #M = self.Ms[d] if d < len(self.Ms) else self.build_M(d)
        M = self.Ms[3]
        d = 3

#         fit_start_time = time.time()
#         fit_func = self.fit_core(f, a, b, n, d, M)
#         fit_end_time = time.time()
#         time_took = fit_end_time - fit_start_time

#         time_left = maxtime - (fit_end_time - start_time)
#         new_points_amount_rel = (b - a) * d
#         estimated_time_new_n = time_took * new_points_amount_rel * 2 + 0.005
#         #print(f"time: took {time_took}s, est {estimated_time_new_n}s, left {time_left}s")
#         estimated_time_new_n = time_left / estimated_time_new_n  #the estimated amount of times we can calculate the new n points in the time left while giving some buffer.
#         n = estimated_time_new_n * new_points_amount_rel
#         #print(f"new n times: {estimated_time_new_n}, n: {n}")
#         n = int(min(new_points_amount_rel * 1000, n))
#         if n >= 2000:
#             time_left = time.time() - start_time
#             fit_func = self.fit_core(f, a, b, n, d, M)
        n = 100
        fit_func = self.fit_core(f, a, b, n, d, M)
        return fit_func

    def fit_core(self, f: callable, a: float, b: float, n: int, d: int, M):
        #x_samples = np.concatenate([[a], (np.random.random(n - 2) * (b - a)) + a, [b]])
        #x_samples.sort()
        x_samples = torch.linspace(a, b, n, dtype=torch.float64)
        y_samples = torch.from_numpy(f(x_samples))

        dis = [0] * n
        def sum_d(acc, i):
            d = acc + euclidean_distance(x_samples[i], y_samples[i], x_samples[i - 1], y_samples[i - 1])
            dis[i] = d
            return d
        d_total = functools.reduce(sum_d, range(1, n), 0)

        t_dis = torch.tensor([dis[i] / d_total for i in range(n)], dtype=torch.float64)
        T = torch.stack([t_dis ** k for k in range(d, -1, -1)]).T
        P = torch.stack([x_samples, y_samples]).T
        C = M.inverse().mm((T.T.mm(T)).inverse()).mm(T.T).mm(P)
#         bezier_curve = M.mm(C).T
        bezier_curve = (T.T.mm(T)).inverse().mm(T.T).mm(P).T
        bezier_x_poly = np.poly1d(convert_torch_to_numpy(bezier_curve[0]))
        bezier_y_poly = np.poly1d(convert_torch_to_numpy(bezier_curve[1]))

        def find_t(x):
            if x == a:
                t = 0
                #print(f"matched x={x} with t={t}")
            elif x == b:
                t = 1
                #print(f"matched x={x} with t={t}")
            else:
                t = find_t_in_interval(x)
            return t
        def find_t_in_interval(x):
            ts = [root for root in (bezier_x_poly - x).roots if root.imag == 0 and -0.15 < root.real < 1.1]
            if len(ts) == 0:
                raise Exception(f"No t_dis matching x={x}")
            elif len(ts) == 1:
                t = ts[0]
                #print(f"matched x={x} with t={t}")
            else:
                estimated_t = (x - a) / (b - a)
                t = min(ts, key=lambda x: abs(x - estimated_t))
                #print(f"{estimated_t}, matched x={x} with {ts}, took {t}")
            return t
        def fit_func(x):
            return bezier_y_poly(find_t(x))

        return fit_func, bezier_curve, bezier_x_poly, bezier_y_poly, T, C, P, M

    def build_M(self, d: int):
        # took code from stackoverflow on how to efficiently calculate a pascal's triangle row
        #see https://stackoverflow.com/questions/15580291/how-to-efficiently-calculate-a-row-in-pascals-triangle
        m = d + 1
        M = torch.zeros([m, m], dtype=torch.float64)

        d_choose_i = 1
        for control_point_index in range(m):
            one_minus_t_power = d - control_point_index
            power_choose_t = 1 if (d + control_point_index) % 2 == 0 else -1
            for choose_1_amount in range(one_minus_t_power + 1):
                M[choose_1_amount][control_point_index] = d_choose_i * power_choose_t
                power_choose_t *= -(one_minus_t_power - choose_1_amount) / (choose_1_amount + 1)
            d_choose_i *= (d - control_point_index) / (control_point_index + 1)
        return M

In [5]:
f = NOISY(1)(poly(1, 1, 1))
a4a = Assignment4A()
t_s = time.time()
a, b = -6, 6
d = 3
fit_func, bezier_curve, bezier_x_poly, bezier_y_poly, T, C, P, M = a4a.fit(f=f, a=a, b=b, d=d, maxtime=5)
t_s = time.time() - t_s

In [6]:
# print(f"M: {M.size()}\n{M}\n")
# print(f"T: {T.size()}\n{T}\n")
# print(f"P: {P.size()}\n{P}\n")
# print(f"C: {C.size()}\n{C}\n")
print(f"Bezier: {bezier_curve.size()}\n{bezier_curve}\n")
print(f"{poly_to_string(bezier_x_poly)}\n{poly_to_string(bezier_y_poly)}")

#get the new curve
pts_real_x = np.linspace(a, b, 2000)
pts_real_y = [fit_func(x) for x in pts_real_x]
pts = T.mm(bezier_curve.T).T
plt.plot(P.T[0],P.T[1],alpha=0.3, color="blue", label="the noisy data")
#plt.scatter(C.T[0],C.T[1], color="red", label="inferred control points")
plt.plot(pts[0], pts[1], color="red", label="the fitted curve")
plt.plot(pts_real_x, pts_real_y, color="green", label="the estimated curve")
plt.legend()
#plt.axis([-1, -0.5, 1.4, 1.8])
plt.show()

Bezier: torch.Size([2, 4])
tensor([[-3.4384e+01,  4.5314e+01, -5.4654e-02, -5.9697e+00],
        [-1.1332e+02,  3.1255e+02, -1.9217e+02,  3.6955e+01]],
       dtype=torch.float64)

-34.38385349938171x^3 + 45.3142674927013x^2 - 0.054654398632967394x - 5.9696891781888715
-113.31549941344895x^3 + 312.55030604096976x^2 - 192.17143066651573x + 36.954620768741776


Exception: No t_dis matching x=-5.99399699849925